In [1]:
!git clone https://github.com/PonyGE/PonyGE2.git

fatal: destination path 'PonyGE2' already exists and is not an empty directory.


In [2]:
!cd PonyGE2/ && git pull

Already up to date.


In [3]:
#pip installs

%pip install numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
%pip install sewar


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
%pip install openpyxl


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import shutil

experiment = "regression"

shutil.move(f"./fitness/{experiment}.py", "./PonyGE2/src/fitness/")


'./PonyGE2/src/fitness/regression.py'

In [7]:
!cd ./PonyGE2/src && python ponyge.py --parameters ../../parameters/{experiment}.txt --dataset_name REDWINE
!cd ./PonyGE2/src & & python ponyge.py --parameters ../../parameters/{experiment}.txt --dataset_name SUNSPOT
!cd ./PonyGE2/src & & python ponyge.py --parameters ../../parameters/{experiment}.txt --dataset_name B1H
!cd ./PonyGE2/src & & python ponyge.py --parameters ../../parameters/{experiment}.txt --dataset_name POLLUTION
!cd ./PonyGE2/src & & python ponyge.py --parameters ../../parameters/{experiment}.txt --dataset_name GAS
!cd ./PonyGE2/src & & python ponyge.py --parameters ../../parameters/{experiment}.txt --dataset_name LAKEERIE
!cd ./PonyGE2/src & & python ponyge.py --parameters ../../parameters/{experiment}.txt --dataset_name Electricity
!cd ./PonyGE2/src & & python ponyge.py --parameters ../../parameters/{experiment}.txt --dataset_name PIGS
!cd ./PonyGE2/src & & python ponyge.py --parameters ../../parameters/{experiment}.txt --dataset_name Nordic
!cd ./PonyGE2/src & & python ponyge.py --parameters ../../parameters/{experiment}.txt --dataset_name CARSALES
shutil.move(f"./PonyGE2/src/fitness/{experiment}.py", "./fitness/")

Traceback (most recent call last):
  File "c:\Users\hsabi\Projects\AIBox\RegressorGE\PonyGE2\src\ponyge.py", line 31, in <module>
    mane()
  File "c:\Users\hsabi\Projects\AIBox\RegressorGE\PonyGE2\src\ponyge.py", line 21, in mane
    set_params(sys.argv[1:])  # exclude the ponyge.py arg itself
    ^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\hsabi\Projects\AIBox\RegressorGE\PonyGE2\src\algorithm\parameters.py", line 262, in set_params
    raise Exception(s)
Exception: algorithm.parameters.set_params
Error: unknown parameters: ['ers/regression.txt', '--dataset_name', 'REDWINE']
You may wish to check the spelling, add code to recognise this parameter, or use --extra_parameters
& was unexpected at this time.
& was unexpected at this time.
& was unexpected at this time.
& was unexpected at this time.
& was unexpected at this time.
& was unexpected at this time.
& was unexpected at this time.
& was unexpected at this time.
& was unexpected at this time.


'./fitness/regression.py'